In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
og_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
og_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = og_df.drop(['EIN', 'NAME', 'STATUS'], axis=1)


df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,N,5000,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   INCOME_AMT              34299 non-null  object
 6   SPECIAL_CONSIDERATIONS  34299 non-null  object
 7   ASK_AMT                 34299 non-null  int64 
 8   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 2.4+ MB


In [4]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_type_counts = df['APPLICATION_TYPE'].value_counts()
cutoff = 100
application_types_to_replace = app_type_counts[app_type_counts < cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [5]:
ask_amt_counts = df['ASK_AMT'].value_counts()
ask_amt_counts.head(30)

,count
ASK_AMT,
5000,25398
6725,3
63981,3
15583,3
10478,3
5170,2
24459,2
35734,2
13499,2


In [6]:
# Choose a cutoff value and create a list of application types to be replaced
ask_amt_counts = df['ASK_AMT'].value_counts()
threshold = 10000

# Identify the values that exceed the threshold
amt_counts_to_replace = ask_amt_counts[ask_amt_counts > threshold].index.tolist()

# Replace values greater than threshold with a new value (e.g., 'Other' or a number)
df.loc[df['ASK_AMT'] > threshold, 'ASK_AMT'] = 'Other'  # Or you can replace with a number, e.g., -1

# Check to make sure replacement was successful
check = df['ASK_AMT'].value_counts()
check

<ipython-input-6-3edcab734a65>:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Other' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['ASK_AMT'] > threshold, 'ASK_AMT'] = 'Other'  # Or you can replace with a number, e.g., -1


,count
ASK_AMT,
5000,25398
Other,8352
6725,3
8181,2
5731,2
...,...
8517,1
8123,1
9725,1


In [7]:
class_counts = df['CLASSIFICATION'].value_counts()
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 70
classifications_to_replace = class_counts[class_counts < cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
Other,499
C1700,287
C4000,194


In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
df_dummies = pd.get_dummies(df)
df_dummies

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ASK_AMT_9894,ASK_AMT_9908,ASK_AMT_9919,ASK_AMT_9931,ASK_AMT_9952,ASK_AMT_9959,ASK_AMT_9971,ASK_AMT_9990,ASK_AMT_10000,ASK_AMT_Other
0,1,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,0,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34295,0,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34296,0,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34297,1,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
# Split our preprocessed data into our features and target arrays
y = df_dummies['IS_SUCCESSFUL']
X = df_dummies.drop('IS_SUCCESSFUL', axis = 1)

In [10]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# Define the model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=75, activation="relu", input_dim=X_train.shape[1]))
nn.add(tf.keras.layers.Dropout(0.2))  # Increase the dropout rate

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=25, activation="relu"))
nn.add(tf.keras.layers.Dropout(0.2))  # Increase the dropout rate


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 75)                  │          42,450 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 75)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 25)                  │           1,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 25)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,376 (173.34 KB)

 Trainable params: 44,376 (173.34 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Compile the model
nn.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [14]:
# Try bigger batch sizes
history = nn.fit(
    X_train_scaled, y_train,
    epochs=100,
    batch_size=35,  # bigger batch size
    validation_data=(X_test_scaled, y_test),
)


Epoch 1/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.6810 - loss: 0.6277 - val_accuracy: 0.7266 - val_loss: 0.5582
Epoch 2/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7316 - loss: 0.5593 - val_accuracy: 0.7268 - val_loss: 0.5571
Epoch 3/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7345 - loss: 0.5485 - val_accuracy: 0.7288 - val_loss: 0.5582
Epoch 4/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7358 - loss: 0.5438 - val_accuracy: 0.7301 - val_loss: 0.5595
Epoch 5/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7309 - loss: 0.5472 - val_accuracy: 0.7319 - val_loss: 0.5588
Epoch 6/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7334 - loss: 0.5454 - val_accuracy: 0.7320 - val_loss: 0.5586
Epoch 7/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7306 - loss: 0.5480 - val_accuracy: 0.7328 - val_loss: 0.5564
Epoch 8/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7364 - loss: 0.5406 - val_accu

In [15]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

322/322 - 1s - 2ms/step - accuracy: 0.7311 - loss: 0.5876
Loss: 0.5875593423843384, Accuracy: 0.7310981750488281
